In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import fiona
from shapely.geometry import Point, shape
from gisutils import df2shp, project
from pydrograph import Nwis

In [ ]:
extent_shp = '../processed_input/extend_15km_buffer.shp'
epsg = 5070

In [ ]:
extent_poly = shape(fiona.open(extent_shp).next()['geometry'])
extent_poly_ll = project(extent_poly, "+init=epsg:{}".format(epsg), "+init=epsg:4269")
extent_poly_ll.bounds

In [ ]:
nwis = Nwis(extent=extent_poly_ll)


In [ ]:
all_dvs = nwis.get_siteinfo('daily_values')

In [ ]:
all_dvs.index = ['{:08d}'.format(i) for i in all_dvs.index]
all_dvs

### Gage 01436690 -- This gage is outside of the model domain. csv export has been commented out (Nick 8/14/2020)

In [ ]:
neversink_inflow = nwis.get_all_dvs(['01436690'])['01436690']

In [ ]:
neversink_inflow['105410_00060_00003'].plot()

In [ ]:
  neversink_inflow.loc['01-01-2010':'12-31-2015']['105410_00060_00003'].quantile(.5)

In [ ]:
  neversink_inflow.loc['01-01-2010':'12-31-2015']['105410_00060_00003'].mean()

In [ ]:
# convert from CFS to m^3/d
# 1cfs = 2446.58 m^3/d

In [ ]:
m3d_neversink_inflow =   neversink_inflow.loc['01-01-2010':'12-31-2015']['105410_00060_00003'].quantile(.5) * 2446.58

In [ ]:
m3d_neversink_inflow

In [ ]:
inflow_df = pd.DataFrame({'datetime':['2011-01-02'],
                         'line_id':[4148040],
                         'flow_m3d':[m3d_neversink_inflow],
                         'category':['NWIS']})
inflow_df


I commented out the export line, I don't think this is the gage we want

In [ ]:
#inflow_df.to_csv('../processed_input/neversink_inflow.csv',index=None)

## Gage 1436000 -- I believe this is the gage we want to use, below Neversink Reservoir

In [ ]:
neversink_inflow = nwis.get_all_dvs(['01436000'])#['01436000']

In [ ]:
neversink_inflow = neversink_inflow['01436000']

In [ ]:
neversink_inflow.head()

In [ ]:
neversink_inflow['105408_00060_00003'].plot()
plt.title('1436000 - full record')
plt.show()
neversink_inflow['105408_00060_00003'].plot(logy=True, lw=0.75)
plt.title('1436000 - full record; LOG Y')
plt.show()

In [ ]:
wood_full_file = '01436500_00060_full.txt'
flow_data_pth = '../../../../data/flow/'
woodbourne_full = pd.read_csv(os.path.join(flow_data_pth, wood_full_file), delim_whitespace=True, skiprows=28, comment='#', parse_dates=[2])
woodbourne_full = woodbourne_full.rename(columns={'105409_00060_00003':'discharge_cfs'})
woodbourne_full = woodbourne_full.drop(columns={'agency_cd', '105409_00060_00003_cd'})
woodbourne_full.head()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(neversink_inflow.index, neversink_inflow['105408_00060_00003'], label='NS inflow')
plt.plot(woodbourne_full.datetime, woodbourne_full.discharge_cfs, lw=0.5, label='NS at Woodbourne')
plt.xlim(pd.to_datetime('1-1-1980'), pd.to_datetime('1-1-2020'))
plt.yscale('log')
plt.legend()
plt.show()

**Want to discuss period over which we consider inflow.** The streamflow record at the gage on the Neversink River that we're using for a flux target (01436500, Neversink near Woodbourne, NY) ends in 1993. Inflows during the period over which our groundwater model is run (2010-2015) are pretty different that the period of record used to generate the flux target (1955-1993). See below:

**2010-01-01 through 2015-12-31**

In [ ]:
neversink_inflow.loc['01-01-2010':'12-31-2015']['105408_00060_00003'].quantile(.5)

In [ ]:
neversink_inflow.loc['01-01-2010':'12-31-2015']['105408_00060_00003'].quantile(.1)

In [ ]:
neversink_inflow.loc['01-01-2010':'12-31-2015']['105408_00060_00003'].mean()

**1954-10-01 through 1993-09-30**

In [ ]:
neversink_inflow.loc['10-01-1954':'09-30-1993']['105408_00060_00003'].quantile(.5)

In [ ]:
neversink_inflow.loc['10-01-1954':'09-30-1993']['105408_00060_00003'].mean()

For consistency previous approach, use 0.5 quantile during 2010-2015 for inflow 

In [ ]:
#  2010-2015
#m3d_neversink_inflow = neversink_inflow.loc['01-01-2010':'12-31-2015']['105408_00060_00003'].quantile(.5) * 2446.58

#  1954-1993
m3d_neversink_inflow = neversink_inflow.loc['10-01-1954':'09-30-1993']['105408_00060_00003'].quantile(.5) * 2446.58

print('neversink inflow: {0:.1f} cfs, {1:.0f} m3/d'.format(
    neversink_inflow.loc['10-01-1954':'09-30-1993']['105408_00060_00003'].quantile(.5), 
    m3d_neversink_inflow
    ))

In [ ]:
inflow_df = pd.DataFrame({'datetime':['2011-01-02'],
                         'line_id':[4148040],
                         'flow_m3d':[m3d_neversink_inflow],
                         'category':['NWIS']})

inflow_df

In [ ]:
inflow_df.to_csv('../processed_input/neversink_inflow.csv',index=None)